In [1]:
import re
import pandas as pd

In [2]:
from tika import parser

# Đọc file doc
file_path = "(VN) Chú giải HS 2022.doc"
parsed = parser.from_file(file_path)

# In nội dung từng dòng
content = parsed["content"]
content_list = content.splitlines()


2025-01-08 17:45:57,546 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [3]:
content_list = [x.strip() for x in content_list if x.strip()]

In [4]:
content_list

['PHỤ LỤC',
 'PHỤ LỤC',
 'CHÚ GIẢI CHI TIẾT DANH MỤC HS2022 (EN2022)',
 '(Ban hành kèm theo công văn số 1810/TCHQ-TXNK ngày 26 tháng 4 năm 2024 của Tổng cục Hải quan)',
 'PHẦN I:',
 'ĐỘNG VẬT SỐNG; CÁC SẢN PHẨM TỪ ĐỘNG VẬT',
 'Chú giải.',
 '1. Trong phần này, khi đề cập đến một giống hoặc một loài động vật, trừ khi có yêu cầu khác, cần phải kể đến cả giống hoặc loài động vật đó còn non.',
 '2. Trừ khi có yêu cầu khác, trong toàn bộ Danh mục này bất cứ đề cập nào liên quan đến các sản phẩm "được làm khô" cũng bao gồm các sản phẩm được khử nước, làm bay hơi hoặc làm khô bằng đông lạnh.',
 'Chương 1:',
 'Động vật sống',
 'Chú giải.',
 '1. Chương này bao gồm tất cả các loại động vật sống trừ:',
 '(a) Cá và động vật giáp xác, động vật thân mềm và động vật thuỷ sinh không xương sống khác, thuộc nhóm 03.01, 03.06, 03.07 hoặc 03.08;',
 '(b) Vi sinh vật nuôi cấy và các sản phẩm khác thuộc nhóm 30.02; và',
 '(c) Động vật thuộc nhóm 95.08.',
 'TỔNG QUÁT',
 'Chương này bao gồm tất cả các loại sinh

In [37]:
section_re = r"^[Pp][Hh][Ầ ầ][Nn]\s+[IVX]+:?$"
chapter_re = r"^chương\s([1-9]|[1-9][0-9]):?$"
heading_re = r"^\d{2}\.\d{2}\s\-\s"

In [34]:
section_start_index = [index for index,text in enumerate(content_list) if re.search(section_re, text)]

In [38]:
chapter_start_index = [index for index,text in enumerate(content_list) if re.search(chapter_re, text.lower())]

In [8]:
heading_start_index = [index for index,text in enumerate(content_list) if re.search(heading_re, text.lower())]

In [9]:
chugiai_re = r"^Chú\sgiải\.?$"


In [10]:
section_indexs = []
for section_start_idx in section_start_index:
    # Lọc các phần tử trong chapter_start_index lớn hơn section_start_idx
    larger_ = [section_end_idx for section_end_idx in chapter_start_index if section_end_idx > section_start_idx]
    if larger_:  
        section_indexs.append([section_start_idx, min(larger_)])

In [11]:
all_section_info = []

for section in section_indexs:
    section_info= {}
    section_content = content_list[section[0]:section[1]]
    # print(section)
    
    section_info['section_name'] =  content_list[section[0]]
    section_info['vi_description'] = content_list[section[0]+1:section[1]]
    section_info['vi_notes'] = []
    for index, text in enumerate(section_content):
        match = re.search(chugiai_re, text)
        if match:
            section_info['section_name'] =  content_list[section[0]]
            section_info['vi_description'] = content_list[section[0]+1:section[0]+index]
            section_info['vi_notes'] = content_list[section[0]+index :section[1]]
    # print(section_info)
    all_section_info.append(section_info)

In [12]:
# Chuyển dữ liệu thành DataFrame
section_df = pd.DataFrame(all_section_info)
section_df['section_name'] = section_df['section_name'].str.replace(":", "", regex=False)
section_df['section_name'] = section_df['section_name'].str.upper()
section_df['vi_description'] = section_df['vi_description'].apply(lambda x: ''.join([i.upper() for i in x]) if isinstance(x, str) else x)
# Ghi DataFrame ra file Excel
section_df.to_excel('vi_sections_notes_2024.xlsx', index=False)

In [13]:
phanchuong1_re = r"^phân\schương\s+[1iI]$"
phanchuong1_index = [index for index,text in enumerate(content_list) if re.search(phanchuong1_re, text.lower())]

thêm logic 
    Nếu giữa 2 chapter_start_idx có phanchuong1_index thì lấy đến phanchuong1_index, nếu không có

In [42]:
chapter_index = []
for idx,chapter_start_idx in enumerate(chapter_start_index):
    if chapter_start_idx < max(heading_start_index):
        next_chapter_start_idx = heading_start_index[idx+1]
        # print(f"chapter_index: from {chapter_start_idx} to {next_chapter_start_idx}")
    else:
        next_chapter_start_idx = len(content_list)
    # Lọc các phần tử trong phanchuong1_index lớn hơn chapter_start_idx và nhỏ hơn next_chapter_start_idx:
    larger_ = [chapter_end_idx for chapter_end_idx in phanchuong1_index if chapter_end_idx > chapter_start_idx and chapter_end_idx < next_chapter_start_idx]
    if larger_:
        chapter_index.append([chapter_start_idx, min(larger_)])
    else:
        # Lọc các phần tử trong heading_start_index lớn hơn chapter_start_idx
        larger_ = [chapter_end_idx for chapter_end_idx in heading_start_index if chapter_end_idx > chapter_start_idx]
        if larger_:
            chapter_index.append([chapter_start_idx, min(larger_)])

In [43]:
chapter_index

[[9, 22],
 [122, 166],
 [275, 299],
 [653, 696],
 [814, 837],
 [973, 983],
 [1032, 1063],
 [1242, 1269],
 [1445, 1475],
 [1610, 1623],
 [1734, 1810],
 [1926, 1952],
 [2274, 2289],
 [2401, 2413],
 [2508, 2559],
 [2916, 2937],
 [3044, 3062],
 [3154, 3172],
 [3220, 3244],
 [3374, 3408],
 [3566, 3580],
 [3715, 3741],
 [3916, 3925],
 [4082, 4098],
 [4167, 4212],
 [4563, 4607],
 [4827, 4855],
 [5110, 5278],
 [7084, 7251],
 [13634, 14005],
 [13937, 14005],
 [14093, 14116],
 [14530, 14616],
 [14868, 15063],
 [15106, 15330],
 [15317, 15330],
 [15442, 15457],
 [15572, 15660],
 [16365, 16636],
 [17028, 17094],
 [17518, 17547],
 [17713, 17740],
 [17833, 17855],
 [17916, 18056],
 [20219, 20411],
 [20279, 20411],
 [20373, 20411],
 [20461, 20659],
 [21127, 21155],
 [21653, 21659],
 [21732, 21742],
 [21918, 21925],
 [22147, 22153],
 [22315, 22367],
 [22522, 22542],
 [22754, 22774],
 [23008, 23020],
 [23106, 23123],
 [23424, 23483],
 [23656, 23695],
 [23804, 23862],
 [24160, 24217],
 [24464, 24501],
 [

In [44]:
all_chapter_info = []
for chapter in chapter_index:
    chapter_info= {}
    chapter_info['chapter_name'] =  content_list[chapter[0]]
    chapter_info['vi_description'] = content_list[chapter[0]+1]
    chapter_info['vi_notes'] = content_list[chapter[0]+2:chapter[1]]
    chapter_info['vi_general'] = []
    
    chapter_content = content_list[chapter[0]:chapter[1]]
    for idx,content in enumerate(chapter_content):
        # print(idx,content)
        if content == 'TỔNG QUÁT':
            # print(idx)
            chapter_info['vi_notes'] = content_list[chapter[0]+2:chapter[0]+idx]
            chapter_info['vi_general'] = content_list[chapter[0]+idx:chapter[1]]


    all_chapter_info.append(chapter_info)

In [46]:
# Chuyển dữ liệu thành DataFrame
chapter_df = pd.DataFrame(all_chapter_info)
chapter_df['chapter_name'] = chapter_df['chapter_name'].str.replace(":", "", regex=False)
chapter_df['chapter_name'] = chapter_df['chapter_name'].str.upper()
# Ghi DataFrame ra file Excel
chapter_df.to_excel('vi_chapters_notes_2024.xlsx', index=False)

In [18]:
section_chapter_start_index = section_start_index + chapter_start_index
section_chapter_start_index.sort()

In [19]:
header_index = []
for idx,header_start_idx in enumerate(heading_start_index):
    if idx < len(heading_start_index)-1:
        next_header_start_idx = heading_start_index[idx+1]
    else:
        next_header_start_idx = len(content_list)
    # Lọc các phần tử trong section_start_index và chapter_start_index lớn hơn header_start_idx và nhỏ hơn next_header_start_idx:
    larger_ = [header_end_idx for header_end_idx in section_chapter_start_index if header_end_idx > header_start_idx and header_end_idx < next_header_start_idx]
    if larger_:
        header_index.append([header_start_idx, min(larger_)])
    else:
        header_index.append([header_start_idx, next_header_start_idx])


In [20]:
all_header_info = []
for header in header_index:
    header_info= {}
    header_info['header_name'] =  content_list[header[0]]
    header_info['notes'] = content_list[header[0]+1 :header[1]]
    all_header_info.append(header_info)

In [21]:
all_header_info_df = pd.DataFrame(all_header_info)
all_header_info_df.insert(1, "hs", all_header_info_df["header_name"].str.split(" - ").str[0].str.strip())
all_header_info_df.insert(1, "description", all_header_info_df.apply(lambda row : row["header_name"][8:], axis=1))

In [22]:
six_digit_heading_re = r"^\d{4}\.\d{2}\s\-\s"
for idx,row in all_header_info_df.iterrows():
    six_digit_heading_indexs = [idx for idx,row in enumerate(row['notes']) if re.search(six_digit_heading_re, row.lower())]
    if six_digit_heading_indexs:
        row['notes'] =  row['notes'][max(six_digit_heading_indexs)+1:]
        all_header_info_df.at[idx, 'notes'] = row['notes']


In [23]:
all_header_info_df['hs'] = all_header_info_df['hs'].astype(str)


In [24]:
all_header_info_df[['hs','description','notes']].to_excel('vi_heading_notes_2024.xlsx', index=False)